1.1 수학과 파이썬 복습

In [4]:
import numpy as np

x = np.array([1,2,3])
print(x.__class__)
print(x.shape)
print(x.ndim)

<class 'numpy.ndarray'>
(3,)
1


In [5]:
W = np.array([[1,2,3], [4,5,6]])
print(W.shape)
print(W.ndim)

(2, 3)
2


In [11]:
W = np.array([[1,2,3], [4,5,6]])
X = np.array([[0,1,2], [3,4,5]])
W + X

array([[ 1,  3,  5],
       [ 7,  9, 11]])

In [12]:
W * X

array([[ 0,  2,  6],
       [12, 20, 30]])

In [13]:
A = np.array([[1,2], [3,4]])
A * 10

array([[10, 20],
       [30, 40]])

In [15]:
b = np.array([10,20])
A * b

array([[10, 40],
       [30, 80]])

In [16]:
a = np.array([1,2,3])
b = np.array([4,5,6])
np.dot(a,b)

32

In [17]:
A = np.array([[1,2], [3,4]])
B = np.array([[5,6], [7,8]])
np.matmul(A,B)

array([[19, 22],
       [43, 50]])

1.2 신경망 추론

In [22]:
import numpy as np

W1 = np.random.randn(2,4)
b1 = np.random.randn(4)
W2 = np.random.randn(4,3)
b2 = np.random.randn(3)
x = np.random.randn(10,2)
h = np.matmul(x, W1) + b1
s = np.matmul(a, W2) + b2

In [20]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [21]:
a = sigmoid(h)